In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import cv2

In [15]:
def load_and_preprocess_data(base_path, img_size=(224, 224)):
    images = []
    depths = []
    labels = []

    color_path = os.path.join(base_path, 'color')
    depth_path = os.path.join(base_path, 'depth')
    
    for img_name in os.listdir(color_path):
        # Load and preprocess the image
        img = cv2.imread(os.path.join(color_path, img_name))
        img = cv2.resize(img, img_size)
        img = img / 255.0  # Normalize to [0, 1]
        
        # Load and preprocess the corresponding depth map
        depth_name = img_name.replace('real', 'depth').replace('fake', 'depth')
        depth = cv2.imread(os.path.join(depth_path, depth_name), cv2.IMREAD_GRAYSCALE)
        depth = cv2.resize(depth, img_size)
        depth = depth / 255.0  # Normalize to [0, 1]
        
        # Append to lists
        images.append(img)
        depths.append(depth)
        
        # Label: 0 for real, 1 for fake
        label = 0 if 'real' in img_name else 1
        labels.append(label)
    
    return np.array(images), np.array(depths), np.array(labels)

# Load the training data
train_img_path = 'C:\Users\Kavyansh Tyagi\OneDrive\Desktop\programming\advanced computer vision\SIH2024\d1\ train_img'
train_images, train_depths, train_labels = load_and_preprocess_data(train_img_path)

# Load the testing data
test_img_path = 'C:\Users\Kavyansh Tyagi\OneDrive\Desktop\programming\advanced computer vision\SIH2024\d1\ test_img'
test_images, test_depths, test_labels = load_and_preprocess_data(test_img_path)



SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1630855613.py, line 32)

In [16]:
# Set the base directory where 'train_img' folder is located
base_dir = r'C:\Users\Kavyansh Tyagi\OneDrive\Desktop\programming\advanced computer vision\SIH2024\d1\train_img'

# Define the paths to 'color' and 'depth' folders
color_folder = os.path.join(base_dir, 'train_img', 'color')
depth_folder = os.path.join(base_dir, 'train_img', 'depth')

# Function to preprocess images and depth maps
def preprocess_images_and_depth_maps(color_folder, depth_folder):
    color_images = []
    depth_images = []
    labels = []

    for filename in os.listdir(color_folder):
        color_path = os.path.join(color_folder, filename)
        depth_path = os.path.join(depth_folder, filename)

        if os.path.exists(color_path) and os.path.exists(depth_path):
            # Read and preprocess the color image
            color_image = cv2.imread(color_path)
            color_image = cv2.resize(color_image, (224, 224))
            color_image = color_image / 255.0  # Normalize to [0, 1]
            color_images.append(color_image)

            # Read and preprocess the depth image
            depth_image = cv2.imread(depth_path, cv2.IMREAD_GRAYSCALE)
            depth_image = cv2.resize(depth_image, (224, 224))
            depth_image = depth_image / 255.0  # Normalize to [0, 1]
            depth_images.append(depth_image)

            # Determine label based on filename (assuming filenames contain 'real' or 'fake')
            if 'real' in filename:
                labels.append(0)
            else:
                labels.append(1)
    
    # Convert lists to numpy arrays
    color_images = np.array(color_images)
    depth_images = np.expand_dims(np.array(depth_images), axis=-1)  # Expand dims for depth
    labels = np.array(labels)

    return color_images, depth_images, labels

# Preprocess the images
color_images, depth_images, labels = preprocess_images_and_depth_maps(color_folder, depth_folder)

# Combine color and depth images
combined_images = np.concatenate([color_images, depth_images], axis=-1)

# Create TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((combined_images, labels))
dataset = dataset.shuffle(buffer_size=1000).batch(32).prefetch(tf.data.AUTOTUNE)

# Sample Output
for images, labels in dataset.take(1):
    print(images.shape)
    print(labels)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Kavyansh Tyagi\\OneDrive\\Desktop\\programming\\advanced computer vision\\SIH2024\\d1\\train_img\\train_img\\color'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.01,
    zoom_range=[0.9, 1.25],
    horizontal_flip=True,
    fill_mode='reflect'
)

train_generator = train_datagen.flow(X_train, y_train, batch_size=32)


In [ ]:
def create_model(input_shape):
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # Fully connected layers
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    
    model.add(layers.Dense(2, activation='softmax'))

    return model

input_shape = (112, 112, 4)  # 3 color channels + 1 depth channel
model = create_model(input_shape)


In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Fit the model on the training data
model.fit(train_generator, epochs=10, validation_data=(X_test, y_test))


In [ ]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')


In [ ]:
# Save the trained model
model.save('face_spoofing_detector_tf.h5')
